
ВАЖНО! Чтобы ваше решение сохранялось и вы могли его нам предоставить в качестве ответа, сохраните себе копию ноутбука (Файл -> Сохранить копию на Диск).

Затем разрешите доступ к ноутбуку по ссылке (Поделиться -> Все у кого есть ссылка)

Вводная информация:

Мы занимаемся автоматизацией управления личных кабинетов клиентов, торгующих на маркетплейсах. В заданиях представлено 5 таблиц, отражающих упрощённые варианты источников, с которыми мы работаем ежедневно для расчёта параметров, составления отчетов, дашбордов и тд. В рамках заданий Вам необходимо проанализировать эти данные и представить результаты по каждой задаче.

С помощью данной тетрадки вы сможете ответить на вопросы тестового задания,используя для это SQL запросы или Python, в зависимости от личных предпочтений.  

В случае, если Вы не владеете навыками работы с SQL или Python, задания можно выполнять используя эксель. В этом случае мы настаиваем на выполнении дополнительных задач.

Для работы Вам будет полезно ознакомиться со структурой базы данных и описанием источников

[Структура БД](https://drive.google.com/file/d/1LRUrIpcgB4Bcv1kpSp-Vv7Gu3r4wTHTy/view?usp=drive_link)

[Описание БД](https://docs.google.com/document/d/1ms9ln6aZDRhM884Ddqr4ZUPQ_sryZ7VMWlX3EpEsuOg/edit?usp=sharing)

# SQL

In [493]:
import gdown
import pandas as pd
import subprocess
import os
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize' : (12, 5)}, font='verdana')
plt.style.use('dark_background')
import sqlite3 as sql3

Подгружаем базу

In [495]:
url='https://drive.google.com/file/d/1hAtTBGUnAQjp3yMIx-i1Qx3QJ3XFjZmv/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
gdown.download(url, 'test_bd_for_sql.db', quiet=False)
connection = sql3.connect('test_bd_for_sql.db')

Downloading...
From: https://drive.google.com/uc?id=1hAtTBGUnAQjp3yMIx-i1Qx3QJ3XFjZmv
To: c:\Users\mail\OneDrive\Documents\my_python\2_ad_hoc\testing_me\test_bd_for_sql.db
100%|██████████| 6.03M/6.03M [00:00<00:00, 20.5MB/s]


Смотрим на таблицы в базе

In [496]:
sql_query = """
            SELECT *
            FROM sqlite_master
            WHERE type = 'table'
            """
cursor = connection.cursor()
cursor.execute(sql_query)
a = cursor.fetchall()
t = []
for i in range(12):
    try:
        t.append(a[i][4].split("\"")[1])
    except IndexError:
        continue
t

['Nomenclature',
 'ComissionBySubjectName',
 'OrdersByDate',
 'WarehouseList',
 'StorageCostByDay']

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Продажами считаем заказы, которые не были отменены и имеют статус "Клиентский"

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную переменную, с которой вы будете работать в дальнейшем.

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

Изучим данные. Посмотрим на Nomenclature

In [497]:
query = """        SELECT * FROM Nomenclature LIMIT 2        """
pd.read_sql_query(query, connection)

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice
0,9428193,0,229720501,YORK,Щетки для стекол,342.92
1,9347354,0,98675218,YORK,Щетки для стекол,260.56


In [498]:
query = """ SELECT COUNT(NmId), COUNT(DISTINCT NmId) FROM Nomenclature   """
pd.read_sql_query(query, connection)

,COUNT(NmId),COUNT(DISTINCT NmId)
0,108,47


NmId не уникален и выполняет фунцию сопоставления в комбинации с SizeName

In [499]:
query = """ SELECT DISTINCT NmId, SizeName FROM Nomenclature ORDER BY NmId """
pd.read_sql_query(query, connection)

,NmId,SizeName
0,1362534,0
1,1407438,0
2,1472848,0
3,1668975,0
4,1927403,0
...,...,...
103,9174701,42
104,9174701,38
105,9326663,0
106,9347354,0


Все комбинации NmId и SizeName уникальны. 

Посмотрим на таблицу OrdersByDate

In [ ]:
query = """ SELECT * FROM  OrdersByDate WHERE SizeName IS NULL """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,3747525,2024-11-16T12:46:11,344.0,0,Клиентский,210284,None
1,3757368,2024-11-16T12:47:47,352.0,0,Клиентский,210284,None
2,2496844,2024-11-16T12:51:41,396.0,0,Клиентский,210284,None
3,5767819,2024-11-16T12:58:37,797.0,0,Клиентский,206348,None
4,5767819,2024-11-16T12:58:37,797.0,0,Клиентский,206348,None
...,...,...,...,...,...,...,...
2010,2496844,2024-11-30T23:47:25,327.0,0,Клиентский,206348,None
2011,7059036,2024-11-30T23:49:11,874.0,0,Клиентский,206348,None
2012,7059036,2024-11-30T23:49:11,874.0,0,Клиентский,206348,None
2013,2496844,2024-11-30T23:59:58,327.0,0,Клиентский,206348,None


5675 записей, из которых у 2017 стоит NULL вместо размера

In [516]:
query = """ 
        SELECT * 
        FROM  Nomenclature as n
        LEFT JOIN 
         (  
        SELECT * 
        FROM  OrdersByDate
        WHERE SizeName IS NULL
        GROUP BY  NmId,SizeName    ) as t  
        ON n.NmId = t.NmId

        WHERE t.SizeName IS NULL AND t.date IS NOT NULL
        """
pd.read_sql_query(query, connection)

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,9428193,0,229720501,YORK,Щетки для стекол,342.92,9428193,2024-11-16T14:31:12,897.00,0,Клиентский,210284,None
1,9347354,0,98675218,YORK,Щетки для стекол,260.56,9347354,2024-11-16T20:24:09,758.00,0,Клиентский,120762,None
2,9326663,0,109595410,YORK,Щетки косметические,90.76,9326663,2024-11-16T15:55:39,301.00,0,Клиентский,206348,None
3,7156668,S,350075652,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
4,7156668,M,350075653,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
5,7156668,L,350075654,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
6,7059036,0,318908541,YORK,Салфетки для уборки,440.80,7059036,2024-11-22T03:15:17,873.00,0,Клиентский,206348,None
7,6032372,0,238666916,YORK,Швабры,820.48,6032372,2024-11-17T13:22:08,2386.00,0,Клиентский,210284,None
8,5767819,0,101177417,YORK,Щетки для уборки,252.65,5767819,2024-11-16T12:58:37,797.00,0,Клиентский,206348,None
9,5717244,0,98674253,YORK,Губки для посуды,49.56,5717244,2024-11-16T14:27:37,373.00,0,Клиентский,206348,None


Видим, что в таблице заказов не заполнена часть  размеров. При этом большая часть NULL - это не указанный 0, кроме артикула ", но мы уже не узнаем какой их размеров был выдан в заказе, поэтому при сборке джойнов все NULL в размерах заменим на размер 0

Посмотрим на таблицу WarehouseList

In [518]:
query = """  SELECT * FROM WarehouseList LIMIT 2 """
pd.read_sql_query(query, connection)

,Origid,Warehouse,WorkTime,NearCityName
0,218987,Алматы Атакент,24/7,Махачкала
1,204939,Астана,24/7,Петропавловск-Камчатский


In [519]:
query = """  SELECT COUNT(Origid), COUNT(DISTINCT Origid) FROM WarehouseList LIMIT 2 """
pd.read_sql_query(query, connection)

,COUNT(Origid),COUNT(DISTINCT Origid)
0,205,121


In [521]:
query = """  SELECT * FROM WarehouseList ORDER BY Origid DESC"""
pd.read_sql_query(query, connection)

,Origid,Warehouse,WorkTime,NearCityName
0,336442,Щербинка,24/7,Москва
1,333530,СЦ Симферополь Молодежненское,24/7,Симферополь
2,324108,Астана 2,24/7,Москва
3,321932,Чашниково,24/7,Москва
4,318294,СЦ Софьино,24/7,Москва
...,...,...,...,...
200,1193,Хабаровск,24/7,Хабаровск
201,686,Новосибирск,24/7,Новосибирск
202,686,Новосибирск,24/7,Новосибирск
203,507,Коледино,24/7,Москва


Часть записей в таблице задвоена, а значит при джойне надо будет убрать дубли

In [541]:
query = """  SELECT DISTINCT Origid, Warehouse FROM WarehouseList ORDER BY Origid DESC"""
pd.read_sql_query(query, connection)

,Origid,Warehouse
0,336442,Щербинка
1,333530,СЦ Симферополь Молодежненское
2,324108,Астана 2
3,321932,Чашниково
4,318294,СЦ Софьино
...,...,...
116,2737,Санкт-Петербург (Уткина Заводь)
117,1733,Екатеринбург - Испытателей 14г
118,1193,Хабаровск
119,686,Новосибирск


Кстати, о складах - в таблице заказов есть отгрузки cо склада с id = 0, тогда как в таблице складов такого склада нет

На 143413 отгрузили неизвестно откуда...

In [550]:
query = """ SELECT * FROM OrdersByDate WHERE warehouseID = 0 AND isCancel = 0 AND orderType="Клиентский" """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,5134688,2024-11-02 21:22:14,636.0,0,Клиентский,0,0
1,5534114,2024-11-07 16:10:13,3890.0,0,Клиентский,0,0
2,5134688,2024-11-07 22:33:04,905.0,0,Клиентский,0,0
3,5134688,2024-11-08 00:59:33,905.0,0,Клиентский,0,0
4,3475731,2024-11-08 20:00:19,711.0,0,Клиентский,0,32
...,...,...,...,...,...,...,...
273,9326663,2024-11-30T14:13:07,301.0,0,Клиентский,0,None
274,9326663,2024-11-30T17:49:41,301.0,0,Клиентский,0,None
275,5717244,2024-11-30T21:26:42,258.0,0,Клиентский,0,None
276,9347354,2024-11-30T22:07:05,636.0,0,Клиентский,0,None


На 143413 отгрузили неизвестно откуда...

In [551]:
query = """ SELECT SUM(SoldPrice) FROM OrdersByDate WHERE warehouseID = 0 AND isCancel = 0 AND orderType="Клиентский" """
pd.read_sql_query(query, connection)

,SUM(SoldPrice)
0,143413.66


Посмотрим на таблицу StorageCostByDay

In [525]:
query = """  SELECT * FROM StorageCostByDay WHERE OfficeId < 0 ORDER BY OfficeId """
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
1,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,0.245385
2,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
3,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,0.245385
4,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,213459505,0.245385
...,...,...,...,...,...,...
947,2024-11-30,-1000033,Пальто,NELIY VINCERE,129045785,2.410800
948,2024-11-30,-1000033,Пальто,NELIY VINCERE,129067756,2.410800
949,2024-11-30,-1000033,Куртки,NELIY VINCERE,241060323,2.410800
950,2024-11-30,-1000033,Куртки,Riches,254332716,2.410800


Склад с отрицательным id...

In [526]:
query = """  SELECT * FROM StorageCostByDay WHERE WarehousePrice < 0 ORDER BY OfficeId """
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
1,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
2,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,213459505,-0.024539
3,2024-11-02,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
4,2024-11-02,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
...,...,...,...,...,...,...
16254,2024-11-30,324108,Футболки,FRANK VANYA,126151327,-0.005075
16255,2024-11-30,324108,Майки спортивные,Flavour knit Gymnastic,213459548,-0.008678
16256,2024-11-30,324108,Купальники гимнастические,Flavour knit Gymnastic,213459615,-0.008678
16257,2024-11-30,324108,Купальники гимнастические,Flavour knit Gymnastic,319883878,-0.017357


... и отрицательные стоимости хранения...

In [528]:
query = """     SELECT * FROM StorageCostByDay WHERE ChrtId = "31789515" ORDER BY OfficeId limit 2"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,507,Водолазки,Flavour knit,31789515,0.1015
1,2024-11-01,507,Водолазки,Flavour knit,31789515,0.5075


... и полные дубли записей с разной стоимостью хранения

In [ ]:
query = """     SELECT *, MIN(WarehousePrice), MAX(WarehousePrice), MAX(WarehousePrice) - MIN(WarehousePrice) as diff
                FROM StorageCostByDay
                GROUP BY ChrtId, Date, OfficeId, brand, Subject
                HAVING diff <> 0             
"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice,MIN(WarehousePrice),MAX(WarehousePrice),diff
0,2024-11-01,507,Водолазки,Flavour knit,31789515,0.507500,0.101500,0.507500,0.406000
1,2024-11-01,120762,Водолазки,Flavour knit,31789515,0.563500,0.161000,0.563500,0.402500
2,2024-11-01,206348,Водолазки,Flavour knit,31789515,0.154000,0.077000,0.154000,0.077000
3,2024-11-02,507,Водолазки,Flavour knit,31789515,0.507500,0.101500,0.507500,0.406000
4,2024-11-02,120762,Водолазки,Flavour knit,31789515,0.563500,0.161000,0.563500,0.402500
...,...,...,...,...,...,...,...,...,...
15034,2024-11-12,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711
15035,2024-11-13,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711
15036,2024-11-14,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711
15037,2024-11-15,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711


Итого имеем вот такие разбросы стоимости хранения для 15039 комбинаций даты, склада, товара, группы и размера. 

Причины такого положения вещей непонятны, поэтому принимаю управленческое решение и в качестве стоимости хранения выбираю максимальную стоимость. Если по факту откажется меньше - хозяин товара только обрадуется

In [535]:
query = """     SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                FROM StorageCostByDay
                GROUP BY ChrtId, Date, OfficeId, brand, Subject"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehouseCost
0,2024-11-19,117501,Костюмы,LAVIRA,24078191,2.975000
1,2024-11-20,117501,Костюмы,LAVIRA,24078191,2.975000
2,2024-11-06,2737,Платья,LAVIRA,25556765,4.704000
3,2024-11-07,2737,Платья,LAVIRA,25556765,4.704000
4,2024-11-08,2737,Платья,LAVIRA,25556765,4.704000
...,...,...,...,...,...,...
51404,2024-11-12,1733,Швабры,YORK,416723700,1.247344
51405,2024-11-13,1733,Швабры,YORK,416723700,1.247344
51406,2024-11-14,1733,Швабры,YORK,416723700,1.247344
51407,2024-11-15,1733,Швабры,YORK,416723700,1.247344


Соберем все вместе, не забыв исключить из записей возвраты и не "клиентские" заказы

In [538]:
query = """
        WITH cte AS
            (SELECT         ord.date as date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.iscancel, ord.orderType, ord.warehouseID,
                            nom.BrandName, nom.SubjectName, nom.CostPrice, nom.ChrtId,
                            com.Commission,  com.CategoryName,
                            war.Warehouse, war.Origid1 as Origid
                    --        st.Date, st.ChrtId,  st.WarehousePrice
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             --WHERE warehouseID >0
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
              --                  LEFT JOIN 
              --                    (SELECT              STRFTIME('%d-%m-%Y', Date) as Date, *
              --                     FROM                StorageCostByDay as ord
              --                     WHERE               WarehousePrice >0
              --                     GROUP BY            ChrtId, Date, OfficeId) as st
              --                    ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
       --     WHERE             iscancel = 0 AND orderType = "Клиентский"
      
                              )
        
        SELECT              *
        FROM                cte
       -- where             WarehousePrice is not null
        order by ChrtId desc
        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,iscancel,orderType,warehouseID,BrandName,SubjectName,CostPrice,ChrtId,Commission,CategoryName,Warehouse,Origid
0,2024-11-04 03:35:36,2752424,0,2043.0,0,Клиентский,206348,Авангард,Полотенца банные,658.3,400676464.0,19.5,Текстиль для дома,Тула,206348.0
1,2024-11-04 23:13:08,2752424,0,2053.0,1,Клиентский,206348,Авангард,Полотенца банные,658.3,400676464.0,19.5,Текстиль для дома,Тула,206348.0
2,2024-11-07 21:51:15,2752424,0,2053.0,0,Клиентский,206348,Авангард,Полотенца банные,658.3,400676464.0,19.5,Текстиль для дома,Тула,206348.0
3,2024-11-08 21:42:21,2752424,0,2177.0,0,Клиентский,206348,Авангард,Полотенца банные,658.3,400676464.0,19.5,Текстиль для дома,Тула,206348.0
4,2024-11-10 15:33:40,2752424,0,2689.0,1,Клиентский,206348,Авангард,Полотенца банные,658.3,400676464.0,19.5,Текстиль для дома,Тула,206348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5670,2024-11-29T13:34:01,7156668,0,344.0,0,Клиентский,117986,None,None,NaN,NaN,NaN,None,Казань,117986.0
5671,2024-11-30T11:28:17,7156668,0,344.0,0,Клиентский,117986,None,None,NaN,NaN,NaN,None,Казань,117986.0
5672,2024-11-30T11:28:17,7156668,0,344.0,0,Клиентский,117986,None,None,NaN,NaN,NaN,None,Казань,117986.0
5673,2024-11-30T11:28:48,7156668,0,344.0,0,Клиентский,117986,None,None,NaN,NaN,NaN,None,Казань,117986.0


In [236]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.iscancel, ord.orderType, 
                            com.CategoryName, nom.BrandName, nom.SubjectName, nom.CostPrice, com.Commission,
                            st.WarehousePrice as storage_cost_by_day, st.Date, st.ChrtId, nom.ChrtId as chr, war.Warehouse
            FROM            OrdersByDate as ord
                              LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId
                              LEFT JOIN ComissionBySubjectName as com 
                                 ON nom.SubjectName = com.SubjectName
                              LEFT JOIN WarehouseList as war
                                 ON war.Origid = ord.warehouseID
                              LEFT JOIN StorageCostByDay as st
                                 ON  st.ChrtId = nom.ChrtId 
                                 -- AND st.Date = ord.date
                                 )
        
        SELECT              *
        FROM                cte 
        WHERE             chr= "416723700"
        --= 4558489
        --LIMIT               5

        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,iscancel,orderType,CategoryName,BrandName,SubjectName,CostPrice,Commission,storage_cost_by_day,Date:1,ChrtId,chr,Warehouse


In [263]:
query = """
        
        SELECT              *
        FROM                OrdersByDate 
       

        """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,4558489,2024-11-02 11:50:46,918.00,1,Клиентский,210284,42-44
1,4558489,2024-11-03 18:30:03,1004.00,1,Клиентский,210284,46-48
2,4558489,2024-11-03 22:12:29,903.14,1,Клиентский,210284,42-44
3,4558489,2024-11-04 02:10:12,903.00,0,Клиентский,210284,46-48
4,4558489,2024-11-04 14:14:54,1421.00,1,Клиентский,210284,54-56
...,...,...,...,...,...,...,...
5670,2496844,2024-11-30T23:47:25,327.00,0,Клиентский,206348,None
5671,7059036,2024-11-30T23:49:11,874.00,0,Клиентский,206348,None
5672,7059036,2024-11-30T23:49:11,874.00,0,Клиентский,206348,None
5673,2496844,2024-11-30T23:59:58,327.00,0,Клиентский,206348,None


In [194]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.iscancel, ord.orderType, nom.ChrtId, 
                            com.CategoryName, nom.BrandName, nom.SubjectName, nom.CostPrice, com.Commission, ord.warehouseID, war.Warehouse, war.WorkTime, war.NearCityName
            FROM            OrdersByDate as ord
                              LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId
                              LEFT JOIN ComissionBySubjectName as com 
                                 ON nom.SubjectName = com.SubjectName
                              LEFT JOIN WarehouseList as war
                                 ON war.Origid = ord.warehouseID)
        
        SELECT              date,
                            NmId,
                            ROUND(AVG(SoldPrice), 2) as average_price, 
                            COUNT(NmId) as units_sold, strftime('%d', MAX(date)) - strftime('%d', MIN(date)) as sales_period,
                            COUNT(NmId)/(strftime('%d', MAX(date)) - strftime('%d', MIN(date))) as sales_speed
        FROM                cte 
        GROUP BY            NmId
        ORDER BY            sales_speed DESC
     --   LIMIT               5

        """
pd.read_sql_query(query, connection)

,date,NmId,average_price,units_sold,sales_period,sales_speed
0,2024-11-01,2496844,356.60,2130,29,73
1,2024-11-01,2572749,1673.68,798,29,27
2,2024-11-01,3176618,739.06,765,29,26
3,2024-11-01,9326663,305.54,723,29,24
4,2024-11-01,3757368,311.97,716,29,24
5,2024-11-01,5767819,624.58,496,29,17
6,2024-11-01,9428193,866.60,431,29,14
7,2024-11-01,9347354,727.02,427,29,14
8,2024-11-01,1362534,561.47,411,29,14
9,2024-11-01,7059036,895.37,356,29,12


In [157]:
query = """
        WITH cte AS
            (SELECT         ord.date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.iscancel, ord.orderType, nom.ChrtId, 
                            com.CategoryName, nom.BrandName, nom.SubjectName, nom.CostPrice, com.Commission, ord.warehouseID, war.Warehouse, war.WorkTime, war.NearCityName
            FROM            OrdersByDate as ord
                              LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId
                              LEFT JOIN ComissionBySubjectName as com 
                                 ON nom.SubjectName = com.SubjectName
                              LEFT JOIN WarehouseList as war
                                 ON war.Origid = ord.warehouseID)
        
        SELECT              MAX(date)-MIN(date),
                            NmId,
                            ROUND(AVG(SoldPrice), 2) as average_price, 
                            COUNT(NmId)
        OVER (PARTITION BY NmId)
        FROM cte 
                GROUP BY NmId
        LIMIT 10
            
        """
pd.read_sql_query(query, connection)


,MAX(date)-MIN(date),NmId,average_price,COUNT(NmId)\n OVER (PARTITION BY NmId)
0,0,1362534,561.47,1
1,0,1407438,992.15,1
2,0,1472848,2404.24,1
3,0,1668975,760.67,1
4,0,2121671,11536.80,1
5,0,2252229,10438.11,1
6,0,2273131,1586.20,1
7,0,2496844,356.60,1
8,0,2572749,1673.68,1
9,0,2752424,2269.84,1


### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа) за месяц, руб.
- рентабельность (маржинальность), %
- медианную прибыль (маржу) за месяц, руб.
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]

Задание 3
### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]


### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]

<b>Вывод</b>
```
Ваш вывод
```



### Дополнительное задание 2
<b>Визуализация</b>
```
Сделать дашборд с основными позателями, которые по Вашему мнению могут понадобиться владельцу кабинета для ежедневной работы
```
Примечание: Выполнение этого задания не обязательно в тетради. Вы можете скачать любую получившуюся у Вас таблицу и выполнить его в любом другом формате любым удобным для Вас способом (PowerBi, xls и др.)
Получившийся файл прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]

```
Ссылка на дашборд
```

# Python

## Python - Загрузка датасетов

In [ ]:
import pandas as pd
import requests
import os
import gdown
from google.colab import files

In [ ]:
!gdown 'https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG'

Downloading...
From: https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG
To: /content/Test_BD_for_Python.xlsx
100% 2.31M/2.31M [00:00<00:00, 72.7MB/s]


In [ ]:
try:
    full_data = pd.ExcelFile('/content/Test_BD_for_Python.xlsx')
    sheet_names = full_data.sheet_names
    print(sheet_names)
    Nomenclature = full_data.parse(sheet_names[0])
    ComissionBySubject = full_data.parse(sheet_names[1])
    OrdersStatistic = full_data.parse(sheet_names[2])
    WarhouseName = full_data.parse(sheet_names[3])
    StorageAmount = full_data.parse(sheet_names[4])
    print("Листы успешно загружены")

except FileNotFoundError:
   print(f"Файл не найден в директории колаба. Пожалуйста, убедитесь, что предыдущая ячейка отработала.")

except Exception as e:
    print(f"Произошла ошибка при загрузке файла: {e}")

['Nomenclature', 'ComissionBySubjectName', 'OrdersByDate', 'WarehouseList', 'StorageCostByDay']
Листы успешно загружены


## Инструкция

После активации блока "Загрузка и подключение к базе данных" база данных была загружена в окружени colab:
~~~
Nomenclature - Справочник номенклатуры
WarehouseList - Справочник складов
OrdersByDate - Статистика по заказам
ComissionBySubjectName - Размер комиссии по категориям
StorageCostByDay - Данные по стоимости хранения
~~~

Сам файл BD появится в меню "Файлы" слева от рабочей области ноутбука. (Test_BD_for_Python.bd)

Отобразить первые/последние n строк можно через функцию display() и head()/tail()

In [ ]:
display(Nomenclature.tail(5))

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice
103,1927403,0,249897385,Azur,Фольга пищевая,29.16
104,1668975,0,235799189,YORK,Щетки кухонные,284.73
105,1472848,0,98674398,YORK,Швабры,1478.18
106,1407438,0,235786280,YORK,Щетки для стекол,394.60
107,1362534,0,258161521,YORK,Щетки косметические,181.52


Для объединения таблиц используйте необходимый тип объединения

```
# outer join
merged_outer = pd.merge(df1, df2, on='nmid', how='outer')
print("Outer join:\n", merged_outer)

# left join
merged_left = pd.merge(df1, df2, on='nmid', how='left')
print("\nLeft join:\n", merged_left)

# right join
merged_right = pd.merge(df1, df2, on='nmid', how='right')
print("\nRight join:\n", merged_right)
```



Сохранить запрос через переменную в эксель файл
~~~
df = pd.DataFrame(Nomenclature)
df.to_excel('/content/Nomenclature.xlsx', sheet_name='Nomenclature', index=False)
~~~

Или напрямую, после реализации сохранения в эксель, дополнив код вызовом загрузчика
~~~
#Скачать сохранённый отчёт из директории
files.download('/content/Nomenclature.xlsx')
~~~

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную переменную, с которой вы будете работать в дальнейшем.

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

In [1]:
#Ваш код

### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа), руб.
- рентабельность (маржинальность), %
- медианную цену заказа
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

In [2]:
#Ваш код

Задание 3
### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

In [3]:
#Ваш код


### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


In [4]:
#Ваш код

<b>Вывод</b>
```
Ваш вывод
```



### Дополнительное задание 2
<b>Визуализация</b>
```
Сделать дашборд с основными позателями, которые по Вашему мнению могут понадобиться владельцу кабинета для ежедневной работы
```
Примечание: Выполнение этого задания не обязательно в тетради. Вы можете скачать любую получившуюся у Вас таблицу и выполнить его в любом другом формате любым удобным для Вас способом (PowerBi, xls и др.)
Получившийся файл прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

In [5]:
#Ваш код

```
Ссылка на дашборд
```

# Excel

Прежде чем перейти к выполнению задания на локальном компьютере, необходимо нажать на кнопку "Выполнить"

<a href="https://imgbb.com/"><img src="https://i.ibb.co/nff863w/image.png" alt="image" border="0"></a>в левом верхнем углу ячейки, вы ее увидите если наведете мышь на соответствующую ячейку, либо воспользоваться комбинацией клавиш `Ctrl+Enter` или `Shift+Enter`

После окончания выполнения кода блока - Вам будет предложено скачать файл в формате xlsx.

Так же нужно ознакомиться с параграфом "Инструкция"  

## Excel - Загрузка файла

In [ ]:
!pip install pandas
!pip install requests

In [ ]:
import pandas as pd
import requests
import os
import gdown
from google.colab import files

In [ ]:
!gdown 'https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG'

Downloading...
From: https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG
To: /content/Test_BD_for_Python.xlsx
100% 2.31M/2.31M [00:00<00:00, 29.9MB/s]


In [ ]:
try:
    full_data = pd.ExcelFile('/content/Test_BD_for_Python.xlsx')
    sheet_names = full_data.sheet_names
    print(sheet_names)
    Nomenclature = full_data.parse(sheet_names[0])
    ComissionBySubject = full_data.parse(sheet_names[1])
    OrdersStatistic = full_data.parse(sheet_names[2])
    WarhouseName = full_data.parse(sheet_names[3])
    StorageAmount = full_data.parse(sheet_names[4])
    print("Листы успешно загружены")

except FileNotFoundError:
   print(f"Файл не найден в директории колаба. Пожалуйста, убедитесь, что предыдущая ячейка отработала.")

except Exception as e:
    print(f"Произошла ошибка при загрузке файла: {e}")

['Nomenclature', 'ComissionBySubjectName', 'OrdersByDate', 'WarehouseList', 'StorageCostByDay']
Листы успешно загружены


In [ ]:
# Скачать сохранённый отчёт из директории
files.download('/content/Test_BD_for_Python.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Инструкция

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную таблицу, с которой вы будете работать в дальнейшем.
Получившийся файл по каждой задаче прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

Ссылка на файл с решением
```
Ваша ссылка
```

### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа), руб.
- рентабельность (маржинальность), %
- медианную цену заказа
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

Ссылка на файл с решением
```
Ваша ссылка
```

### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

Ссылка на файл с решением
```
Ваша ссылка
```


### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


<b>Вывод</b>
```
Ваш вывод
```



### Дополнительное задание 2
<b>Визуализация</b>
```
Сделать дашборд с основными позателями, которые по Вашему мнению могут понадобиться владельцу кабинета для ежедневной работы
```
Примечание: Выполнение этого задания не обязательно в тетради. Вы можете скачать любую получившуюся у Вас таблицу и выполнить его в любом другом формате любым удобным для Вас способом (PowerBi, xls и др.)
Получившийся файл прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

```
Ссылка на дашборд
```